# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [153]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import nltk
import re
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /Users/pritish/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/pritish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/pritish/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [154]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
df = df.dropna(subset=df.select_dtypes(float).columns, how='all')
X = df.message.values
Y = df[df.columns[4:]].values
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [155]:
special_char_regex = '[^\w\*]'
def tokenize(text):
    special_chars = re.findall(special_char_regex, text)
    for special_char in special_chars:
        text = text.replace(special_char, " ")
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [156]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(KNeighborsClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [157]:
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(X, Y)
#vect = CountVectorizer(tokenizer=tokenize)
#tfidf = TfidfTransformer()
#clf = MultiOutputClassifier(KNeighborsClassifier())

# train classifier
model = pipeline
#X_train_counts = vect.fit_transform(X_train)
#X_train_tfidf = tfidf.fit_transform(X_train_counts)
model.fit(X_train, y_train)

#accuracy = (y_pred == y_test).mean()
#print("Accuracy:", accuracy)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x181950790>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [158]:
# predict on test data
#X_test_counts = vect.transform(X_test)
#X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = model.predict(X_test)
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: 0.9163656559489893


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [159]:
for i in range(36):
        print("Statistics for category: ", i, "\n", classification_report(y_test[:,i],y_pred[:,i]))
        
#from sklearn.metrics import classification_report
#print(classification_report(y_test, y_pred))

Statistics for category:  0 
               precision    recall  f1-score   support

         0.0       0.34      0.12      0.17      1544
         1.0       0.77      0.93      0.84      4953
         2.0       0.67      0.04      0.07        55

    accuracy                           0.73      6552
   macro avg       0.59      0.36      0.36      6552
weighted avg       0.66      0.73      0.68      6552

Statistics for category:  1 
               precision    recall  f1-score   support

         0.0       0.84      0.95      0.89      5429
         1.0       0.32      0.11      0.16      1123

    accuracy                           0.81      6552
   macro avg       0.58      0.53      0.53      6552
weighted avg       0.75      0.81      0.77      6552

Statistics for category:  2 
               precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6514
         1.0       0.00      0.00      0.00        38

    accuracy                          

/Users/pritish/anaconda3/envs/ml/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Statistics for category:  18 
               precision    recall  f1-score   support

         0.0       0.87      0.98      0.92      5707
         1.0       0.19      0.02      0.04       845

    accuracy                           0.86      6552
   macro avg       0.53      0.50      0.48      6552
weighted avg       0.78      0.86      0.81      6552

Statistics for category:  19 
               precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      6077
         1.0       0.00      0.00      0.00       475

    accuracy                           0.93      6552
   macro avg       0.46      0.50      0.48      6552
weighted avg       0.86      0.93      0.89      6552

Statistics for category:  20 
               precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6245
         1.0       0.00      0.00      0.00       307

    accuracy                           0.95      6552
   macro avg       0.48      0.50  

### 6. Improve your model
Use grid search to find better parameters. 

In [160]:
# Grid Search to tune hyper-parameters

#print(MultiOutputClassifier(KNeighborsClassifier()).get_params().keys())
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False)
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
model = cv
model.fit(X_train, y_train)

#accuracy = (y_pred == y_test).mean()
#print("Accuracy:", accuracy)

y_pred = model.predict(X_test)


for i in range(36):
        print("Statistics for category: ", i, "\n", classification_report(y_test[:,i],y_pred[:,i]))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
from sklearn.externals import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(model, 'ml_model.pkl') 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.